# Birdsongs 07.- Preprocesando Datos - Cortando Datos


Permite, a partir de un directorio de datos de audios ya transformados según el notebook **Birdsongs_06_Preprocesando_Datos_Convirtiendo_Audio_a_Datos**, que corresponden a un espectrograma o periodograma, dividir estos en matrices de menor tamaño.

La estructura del repositorio de destino se mantiene respecto al de origen, un directorio por cada especie tratada.

![directorio](./resources/directorioaudios.png)

El resultado de este notebook es un repositorio de destino con los datos dividos en arrays de menor longitud.


## 1.- Librerías

Cargamos las librerías de uso común en el notebook

In [1]:
# importar librerías
import numpy as np
import os
import random


## 2.- Funciones

Cargamos las funciones utilizadas en el notebook

### Recupera directorios

Los ficheros de datos se encuentran localizados en un directorio raíz, y dentro del correspondiente subdirectorio al que pertenece la especie. Realizando un dir de los directorios contenidos en el directorio raíz, tendremos una lista con todas las especies que forman parte del dataset


In [4]:
#----------------------------------------------------------------------------
# get_specie_names(path)
#  argumentos: 
#      path: directorio de audios
#----------------------------------------------------------------------------
def get_specie_names(path, size):  
    specie_names = os.listdir(path)
    
    if size != 0:
        specie_names = random.sample(specie_names, size)
        
    print(specie_names)
    
    return specie_names

### Procesa ficheros de datos

Va procesando los ficheros de datos que existen en cada subdirectorio y va creando un directorio de datos, con los ficheros numpy conteniendo la información de la grabación. Hay que especificar:

* Tamaños del tramo
* Tamaño del salto entre tramos
* Número de especies


In [6]:
#----------------------------------------------------------------------------
# split_data(datapath, splitpath)
#  argumentos: 
#      datapath: repositorio de datos origen
#      splitpath: repositorio de datos divididos destino
#----------------------------------------------------------------------------
def split_data(datapath, splitpath):
    # tamaños del split y salto entre splits (100 equivale aproximadamente a 1 segundo)
    splitsize = 100 
    splitskip = 100
    
    
    # número de especies (si se deja sin informar trata todas las especies del directorio)
    size = 0
    
    # crea el directorio raiz de datos si no existe
    if not os.path.exists(splitpath):
        os.mkdir(splitpath)

    # recupera la lista de especies (directorios) a tratar
    specie_names = get_specie_names(datapath, size)
    number_species = len(specie_names)
   
    # itera sobre cada directorio y va troceando cada una de las canciones
    for idx, specie_name in enumerate(specie_names):
        # crea directorio destino de la especie, si no existe
        splitdir = os.path.join(splitpath, specie_name)
        if not os.path.exists(splitdir):
            os.mkdir(splitdir)

        # recupera los ficheros de datos existentes para esta especie
        specie_dir = os.path.join(datapath, specie_name)
        specie_files = os.listdir(specie_dir)
        number_files = len(specie_files)
        number_load = number_files
        
        print(' Specie name = {:14s} - {:3d}'.format(specie_name,idx),
               ", ",number_files," files in this specie", sep=" ", end='\r', flush=True)

        # itera sobre la lista de ficheros de dato recuperados
        printevery = 20
        
        for idx2, infilename in enumerate(specie_files):
            # fichero datos origen
            file_path = specie_dir + '/' + infilename
            file = np.load(file_path)

            # itera sobre el fichero y genera ficheros de tamaño N
            pointer = 0
            counter = 0
            file_length = file.shape[1]
                
            while (pointer < file_length):
                counter += 1

                # nombre del fichero a generar
                splitfile = splitdir + '/' + infilename[:infilename.find('.')] + \
                            '_' + str(counter) +'.npy'
                
                # controlamos que no lleguemos al final del array y el tamaño restante no cuadre
                # con los límites. En este caso, nos quedamos con el final, aunque se puedan dar
                # casos de solapamiento
                if ((pointer + splitsize) > file_length):
                    splitgram = file[:,pointer:file_length]
                else:
                    splitgram = file[:,pointer:pointer + splitsize]
                    
                # graba fichero numpy
                np.save(splitfile, splitgram)
                
                # actualiza puntero de avance, saltando n valores
                pointer = pointer + splitskip
                
            # muestra avance
            if (0 == idx2 % printevery):
                print('\r Loading specie: {:14s} ({:2d} of {:2d} species)'.format(specie_name,idx+1,number_species),
                       ", file ",idx2+1," of ",number_load,": ",file_path, sep=" ", end='\r', flush=True)


## 3.- Genera muestras de menor tamaño

Procesa directorio con los datos en formato numpy y los divide en arrays de menor tamaño, salvándolos en un directorio distinto.


In [7]:
# directorio origen con los datos
datapath= './data/mel'

# directorio destino con los datos divididos
splitpath = './data/meld'

# procesa datos
split_data(datapath, splitpath)


['Anthus trivialis', 'Emberiza citrinella', 'Turdus iliacus', 'Parus major', 'Spinus spinus', 'Lullula arborea', 'Emberiza cirlus', 'Phylloscopus trochilus', 'Phoenicurus phoenicurus', 'Corvus corone']
